# TESS Exoplanet Transit Light Curve Analysis

This notebook analyzes photometric time-series data from NASA's **Transiting Exoplanet Survey Satellite (TESS)** mission.  
Target: **WASP-39b** — a well-characterized hot Jupiter and one of the first targets analyzed by JWST.

### Pipeline Overview
1. Download TESS light curve via `Lightkurve`
2. Detrend systematics (Savitzky–Golay filter + sigma clipping)
3. Find orbital period with Box Least Squares (BLS) periodogram
4. Fit transit model using `batman` (planet radius, inclination, limb darkening)
5. Visualize results

In [ ]:
# Install dependencies (uncomment if needed)
# !pip install lightkurve batman-package scipy matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import lightkurve as lk
import batman
from scipy.optimize import minimize

%matplotlib inline
plt.rcParams['figure.dpi'] = 120
print('All imports successful ✓')

## 1. Download TESS Light Curve

In [ ]:
TARGET = 'WASP-39'

search = lk.search_lightcurve(TARGET, mission='TESS', author='SPOC', exptime=120)
print(search)

lc_raw = search[0].download()
lc_raw.plot(title=f'{TARGET} — Raw TESS Light Curve')

## 2. Preprocessing & Systematic Detrending

In [ ]:
# Select PDCSAP flux (pre-corrected for spacecraft systematics by SPOC pipeline)
lc = lc_raw.select_flux('pdcsap_flux').remove_nans().remove_outliers(sigma=5).normalize()

# Flatten: fits and divides out a Savitzky-Golay polynomial to remove slow trends
# window_length=401 cadences (~13 hours at 2-min cadence) — longer than any transit
lc_flat, trend = lc.flatten(window_length=401, return_trend=True)

fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)
lc.plot(ax=axes[0], label='Normalized')
trend.plot(ax=axes[0], label='SG Trend', color='red')
lc_flat.plot(ax=axes[1], label='Detrended')
axes[0].legend()
plt.tight_layout()

print(f'RMS scatter: {np.std(lc_flat.flux.value)*1e6:.1f} ppm')

## 3. BLS Periodogram — Period Detection

In [ ]:
pg = lc_flat.to_periodogram(method='bls',
                             period=np.linspace(0.5, 15, 10000),
                             duration=[0.05, 0.1, 0.15, 0.2])
pg.plot()

period   = pg.period_at_max_power.value
t0       = pg.transit_time_at_max_power.value
duration = pg.duration_at_max_power.value
depth    = pg.depth_at_max_power

print(f'Best period:    {period:.4f} d  (literature: 4.0552 d)')
print(f'Transit epoch:  {t0:.4f} BTJD')
print(f'Duration:       {duration*24:.2f} hours')
print(f'Depth:          {depth*1e6:.0f} ppm  →  Rp/Rs ≈ {np.sqrt(depth):.4f}')

## 4. batman Transit Model Fit

In [ ]:
def batman_lc(t, t0, period, rp, a, inc, u1, u2):
    """Compute batman transit light curve."""
    p = batman.TransitParams()
    p.t0 = t0; p.per = period; p.rp = rp; p.a = a; p.inc = inc
    p.ecc = 0.0; p.w = 90.0; p.limb_dark = 'quadratic'; p.u = [u1, u2]
    return batman.TransitModel(p, t).light_curve(p)

# Phase-fold the light curve
lc_fold = lc_flat.fold(period=period, epoch_time=t0)

# Fit only points near transit
mask  = np.abs(lc_fold.time.value) < 2 * duration
t_fit = lc_fold.time.value[mask]
f_fit = lc_fold.flux.value[mask]
e_fit = np.ones_like(f_fit) * 1e-4

def neg_log_like(x):
    rp, a, inc, u1, u2 = x
    if rp < 0 or rp > 1 or a < 1 or inc < 50: return 1e10
    model = batman_lc(t_fit, 0.0, period, rp, a, inc, u1, u2)
    return 0.5 * np.sum(((f_fit - model) / e_fit)**2)

# Optimize starting from WASP-39b literature values
result = minimize(neg_log_like, [0.145, 11.4, 87.8, 0.4, 0.2],
                  method='Nelder-Mead',
                  options={'maxiter': 10000, 'xatol': 1e-6})

rp, a, inc, u1, u2 = result.x
print(f'Fitted Parameters:')
print(f'  Rp/Rs        = {rp:.4f}  (lit: 0.1454)')
print(f'  a/Rs         = {a:.2f}   (lit: 11.55)')
print(f'  Inclination  = {inc:.2f}° (lit: 87.83°)')
print(f'  Limb dark u1 = {u1:.3f}')
print(f'  Limb dark u2 = {u2:.3f}')
print(f'  Transit depth = {rp**2*1e6:.0f} ppm')

## 5. Phase-Folded Light Curve + Model

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

# All folded points
ax.scatter(lc_fold.time.value, lc_fold.flux.value,
           s=1, c='grey', alpha=0.3, label='All cadences')

# Binned
binned = lc_fold.bin(time_bin_size=0.01)
ax.errorbar(binned.time.value, binned.flux.value,
            yerr=binned.flux_err.value, fmt='o', ms=4,
            color='steelblue', label='Binned (14.4 min)', zorder=5)

# batman model
t_model = np.linspace(t_fit.min(), t_fit.max(), 1000)
f_model = batman_lc(t_model, 0.0, period, rp, a, inc, u1, u2)
ax.plot(t_model, f_model, 'r-', lw=2,
        label=f'batman fit  Rp/Rs={rp:.4f}', zorder=6)

ax.set_xlabel('Phase (days from mid-transit)')
ax.set_ylabel('Normalized Flux')
ax.set_title('WASP-39b Phase-Folded Transit — batman Model Fit')
ax.legend()
plt.tight_layout()
plt.savefig('transit_fit.png', dpi=150)
plt.show()
print('Saved → transit_fit.png')